In [6]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium
from collections import defaultdict

# 定义请求的URL
url = "https://127.0.0.1:9200/vic-weather/_search"

# 定义请求头
headers = {
    'Content-Type': 'application/json'
}

# 定义请求数据
data = {
    "size": 100,  # 增加查询结果的数量，以确保包含所有地点的最新记录
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "timestamp": {
                "order": "desc"
            }
        }
    ]
}

# 发送GET请求
response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)

# 解析JSON响应
response_json = response.json()

# 从响应JSON数据中提取需要的字段
hits = response_json['hits']['hits']

# 创建一个字典来存储每个地理位置的最新记录
latest_records = {}
for hit in hits:
    source = hit['_source']
    coords = (source['geo']['lat'], source['geo']['lon'])
    if coords not in latest_records:
        latest_records[coords] = source

# 创建一个列表来存储GeoJSON数据
features = []
for coords, source in latest_records.items():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [coords[1], coords[0]]  # 注意：GeoJSON 使用 [lon, lat] 顺序
        },
        "properties": {
            "stationid": source['stationid'],
            "timestamp": source['timestamp'],
            "name": source['name'],
            "air_temp": source['air_temp'],
            "apparent_t": source['apparent_t'],
            "rel_hum": source['rel_hum'],
            "rain_trace": source['rain_trace'],
            "wind_dir": source['wind_dir'],
            "wind_spd_kmh": source['wind_spd_kmh']
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# 使用folium进行地理数据可视化
# 创建地图对象
m = folium.Map(location=[-37.8, 145.0], zoom_start=8)

# 添加标记到地图，并添加弹出窗口显示温度信息
for feature in geojson_data['features']:
    lon, lat = feature['geometry']['coordinates']
    popup_content = f"""
    <b>Station:</b> {feature['properties']['name']}<br>
    <b>Temperature:</b> {feature['properties']['air_temp']} °C<br>
    <b>Apparent Temperature:</b> {feature['properties']['apparent_t']} °C<br>
    <b>Humidity:</b> {feature['properties']['rel_hum']}%<br>
    <b>Wind Direction:</b> {feature['properties']['wind_dir']}<br>
    <b>Wind Speed:</b> {feature['properties']['wind_spd_kmh']} km/h<br>
    <b>Timestamp:</b> {feature['properties']['timestamp']}
    """
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)

# 添加图层控制
folium.LayerControl().add_to(m)

# 保存地图到文件
m.save('weather_map.html')

# 如果在Jupyter Notebook中运行，可以直接显示地图
m


/home/charles/anaconda3/envs/ccc/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
